### **Imports**

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from robolab_turtlebot import Turtlebot
from TurtleControllers import TurtlebotController
from TurtleVision import TurtlebotVision
import TurtleUtils

## **Initialization**

In [2]:
turtle = Turtlebot(rgb = True, pc = True)
turtle_controller = TurtlebotController(turtle, rate = 40)
turtle_vision = TurtlebotVision(turtle)

#### **Find garage center**

In [ ]:
def reg_mean(regs):
    s, n = 0.0, len(regs) * 2
    for reg in regs:
        s += reg[0] + reg[1]
    return s / n

def reg_mid(reg):
    return (reg[0] + reg[1]) / 2

# Find Garage center
while True:
    regs = turtle_vision.get_regions(color = "purple", minimal_size=15)
    
    # Want to see two purple pillars
    while len(regs) < 2:
        turtle_controller.cmd_velocity(angular = 0.5)
        regs = turtle_vision.get_regions(color = "purple", minimal_size=15)
        turtle_controller.rate.sleep()
        
    # Want to look at the center of the garage entrance
    m = reg_mean(regs)
    err = m - turtle_vision.img_width // 2
    print(err)
    if abs(err) < 10:
        turtle_controller.cmd_velocity(linear = 0, angular = 0)
        break
    else:
        if err < 0.0:
            err = np.clip(err, -turtle_controller.max_yaw_rate, -turtle_controller.min_yaw_rate)
        else:
            err = np.clip(err, turtle_controller.min_yaw_rate, turtle_controller.max_yaw_rate)
        turtle_controller.cmd_velocity(linear = 0, angular = -err)
        
    turtle_controller.rate.sleep()

#### **Generate a garage model**

In [ ]:
turtle_controller.reset_odometry()
P1, P2, P_G, P_PG, P_GOAL = turtle_vision.garage_entry_waypoints()

fig, ax = plt.subplots(1)
TurtleUtils.visualize_garage(ax, P1, P2, P_G, P_PG, P_GOAL)

In [ ]:
turtle_controller.move_to(point, relative = False)

In [ ]:
def reg_mean(regs):
    s, n = 0.0, len(regs) * 2
    for reg in regs:
        s += reg[0] + reg[1]
    return s / n

def reg_mid(reg):
    return (reg[0] + reg[1]) / 2

# Find Garage center
while True:
    regs = turtle_vision.get_regions(color = "purple", minimal_size=13)
    
    # Want to see two purple pillars
    while len(regs) < 2:
        turtle_controller.cmd_velocity(angular = 0.5)
        regs = turtle_vision.get_regions(color = "purple", minimal_size=13)
        turtle_controller.rate.sleep()
        
    # Want to look at the center of the garage entrance
    m = reg_mean(regs)
    
    err = m - turtle_vision.img_width // 2
    print(err)
    if abs(err) < 10:
        turtle_controller.cmd_velocity(linear = 0, angular = 0)
        break
    else:
        if err < 0.0:
            err = np.clip(err, -turtle_controller.max_yaw_rate, -turtle_controller.min_yaw_rate)
        else:
            err = np.clip(err, turtle_controller.min_yaw_rate, turtle_controller.max_yaw_rate)
        turtle_controller.cmd_velocity(linear = 0, angular = -err)
        
    turtle_controller.rate.sleep()
    
turtle_controller.reset_odometry()
P1, P2, P_G, P_PG, P_GOAL = turtle_vision.garage_entry_waypoints()

turtle_controller.move_to(P_PG, relative = False)

# Find Garage center
while True:
    regs = turtle_vision.get_regions(color = "purple", minimal_size=13)
    
    # Want to see two purple pillars
    while len(regs) < 2:
        turtle_controller.cmd_velocity(angular = 0.5)
        regs = turtle_vision.get_regions(color = "purple", minimal_size=13)
        turtle_controller.rate.sleep()
        
    # Want to look at the center of the garage entrance
    m = reg_mean(regs)
    
    err = m - turtle_vision.img_width // 2
    print(err)
    if abs(err) < 10:
        turtle_controller.cmd_velocity(linear = 0, angular = 0)
        break
    else:
        if err < 0.0:
            err = np.clip(err, -turtle_controller.max_yaw_rate, -turtle_controller.min_yaw_rate)
        else:
            err = np.clip(err, turtle_controller.min_yaw_rate, turtle_controller.max_yaw_rate)
        turtle_controller.cmd_velocity(linear = 0, angular = -err)
        
    turtle_controller.rate.sleep()
    
turtle_controller.reset_odometry()
P1, P2, P_G, P_PG, P_GOAL = turtle_vision.garage_entry_waypoints()

for point in (P_PG, P_GOAL):
    turtle_controller.move_to(point, relative = False)

In [ ]:
turtle_controller.reset_odometry()
P1, P2, P_G, P_PG, P_GOAL = turtle_vision.garage_entry_waypoints()

for point in (P_PG, P_GOAL):
    turtle_controller.move_to(point, relative = False)

In [5]:
import numpy as np
import matplotlib.pyplot as plt

from robolab_turtlebot import Turtlebot
from TurtleControllers import TurtlebotController
from TurtleVision import TurtlebotVision
import TurtleUtils

#Chceš-li přehledný a čitelný kód, napiš ho v pythnu - Kat master 2022

class turtle_dif_1pyco:
    def __init__(self, turtle_controller, turtle_vision):
        self.turtle_controller = turtle_controller
        self.turtle_vision = turtle_vision

    def main_pyco(self, number_of_calibrations):
        error = False
        if number_of_calibrations < 1:
            print("Pyco zadej přirozené číslo")
            return error
        if number_of_calibrations == 1:
            P_PG, P_GOAL = self.find_garage()
        else:
            for i in range(number_of_calibrations - 1):
                P_PG, P_GOAL = self.find_garage()

        for point in (P_PG, P_GOAL):
            self.turtle_controller.move_to(point, relative = False)

    def reg_mean(self, regs):
        s, n = 0.0, len(regs) * 2 #s = suma, n = pocet objektu
        for reg in regs:
            s += reg[0] + reg[1]
        average = s/n #prumer
        return average

    def reg_mid(self, reg):
        return (reg[0] + reg[1]) / 2

    def find_garage(self):
        finish = False
        while not finish:
            purple_regs = self.turtle_vision.get_regions(color = "purple", minimal_size = 8)
            purple_regs_2 = self.find_2_purple_regs(purple_regs)
            centre_of_garage = self.reg_mean(purple_regs_2)
            err = centre_of_garage - self.turtle_vision.img_width // 2
            finish = self.good_direction(err)
        self.turtle_controller.reset_odometry()
        P1, P2, P_G, P_PG, P_GOAL = self.turtle_vision.garage_entry_waypoints()
        self.turtle_controller.move_to(P_PG, relative = False)
        return P_PG, P_GOAL

    def find_2_purple_regs(self, purple_regs):
        while len(purple_regs) < 2:
            self.turtle_controller.cmd_velocity(angular=0.4)
            purple_regs = self.turtle_vision.get_regions(color="purple", minimal_size=8)
            self.turtle_controller.rate.sleep()
        return purple_regs

    def good_direction(self, err):
        finish = False
        if abs(err) < 10:
            self.turtle_controller.cmd_velocity(linear = 0, angular = 0)
            finish = True
        else:
            if err < 0.0:
                err = np.clip(err, -self.turtle_controller.max_yaw_rate,
                              -self.turtle_controller.min_yaw_rate)
            else:
                err = np.clip(err, self.turtle_controller.min_yaw_rate,
                              self.turtle_controller.max_yaw_rate)
            self.turtle_controller.cmd_velocity(linear = 0, angular = -err)
        self.turtle_controller.rate.sleep()
        return finish

turtle = Turtlebot(rgb = True, pc = True)
turtle_controller = TurtlebotController(turtle, rate = 40)
turtle_vision = TurtlebotVision(turtle)
turtle_dif_1pyco = turtle_dif_1pyco(turtle_controller, turtle_vision)
turtle_dif_1pyco.main_pyco(1)

/home.nfs/jonjaku1/lar/22_3_2023/lar/folder/TurtleVision.py:48: RuntimeWarning: Mean of empty slice
  column_mean = np.nanmean(depth_image_strip, axis=0)
